In [1]:
from sklearn.preprocessing import FunctionTransformer
import numpy as np
import pandas as pd
import yaml

In [2]:
%run ./src/database.py

# %run ./src/data_preprocessing.py

In [28]:
YAML_FILEPATHNAME = "./config.yaml"
PRE_CRUISE_DB = 0
POST_CRUISE_DB = 1

# Preprocessing

In [29]:
# Read YAML file
data_path = read_yaml(YAML_FILEPATHNAME)

In [34]:
# Read Pre_cruise Data
ds_pre_cruise = Database(DATA_PATH)
df_pre_cruise = ds_pre_cruise.db_read(yaml_data['databases'][PRE_CRUISE_DB])

In [35]:
# Read Post_cruise Data
ds_post_cruise = Database(DATA_PATH)
df_post_cruise = ds_post_cruise.db_read(yaml_data['databases'][POST_CRUISE_DB])

In [38]:
%run ./src/database.py
%run ./src/util.py

# Merge Pre_cruise and Post_cruise data
df_cruise = merge_dataframe (df_pre_cruise, df_post_cruise)

In [39]:
df_cruise.shape

(130875, 25)

In [ ]:
def read_pre_cruise_data(data_path, file_name, table_name, index_name):
    return read_db(data_path, file_name, table_name,index_name)

In [ ]:
def read_post_cruise_data(DATA_PATH, "cruise_pre.db", "cruise_pre","index"):
    return read_db(DATA_PATH, "cruise_pre.db", "cruise_pre","index")

In [60]:
# Read df_cruise_pre data file
df_cruise_pre = 

In [27]:
# Sort df_cruise_pre by index
df_cruise_pre = df_cruise_pre.sort_index()

In [28]:
# Under df_cruise_pre data
df_cruise_pre.shape

(133746, 18)

In [29]:
### There are 133746 rows with 18 columns

In [24]:
df_cruise_post = read_db(DATA_PATH, "cruise_post.db", "cruise_post","index")
df_cruise_post = df_cruise_post.sort_index()

In [26]:
df_cruise_post.head(5)

,Cruise Name,Ticket Type,Cruise Distance,Ext_Intcode,WiFi,Dining,Entertainment
index,,,,,,,
0,Blastoise,None,3567 KM,LB446RWOOZI,1.0,1,1.0
0,Blastoise,None,3567 KM,LB446RWOOZI,1.0,1,1.0
0,Blastoise,None,3567 KM,LB446RWOOZI,1.0,1,1.0
0,Blastoise,None,3567 KM,LB446RWOOZI,1.0,1,1.0
1,Blastoise,Deluxe,672 KM,LB138HKBECM,NaN,0,1.0


In [41]:
df_cruise_post.shape

(401261, 7)

In [42]:
### There are 401261 with 7 columns

In [43]:
df_cruise_post = df_cruise_post.drop_duplicates()

In [44]:
df_cruise_post.shape

(130875, 7)

In [45]:
### After removing duplicates there are 130875 rows and 7 columns

In [47]:
df_cruise = pd.merge(df_cruise_pre, df_cruise_post, left_index=True, right_index=True, how='inner')

In [49]:
df_cruise.info()

<class 'pandas.core.frame.DataFrame'>
Index: 130875 entries, 0 to 133745
Data columns (total 25 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   Gender                                      117691 non-null  object 
 1   Date of Birth                               116495 non-null  object 
 2   Source of Traffic                           130875 non-null  object 
 3   Onboard Wifi Service                        111800 non-null  object 
 4   Embarkation/Disembarkation time convenient  115568 non-null  float64
 5   Ease of Online booking                      112945 non-null  float64
 6   Gate location                               114087 non-null  float64
 7   Logging                                     130875 non-null  object 
 8   Onboard Dining Service                      114404 non-null  object 
 9   Online Check-in                             115529 non-null  float64
 10  C

# Feature Engineering

# Model Building

# Model Evaluation